In [ ]:
pip install -r requirements.txt

In [ ]:
import os
from os import environ
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Load credentials
os.environ["OPENAI_API_KEY"] = os.getenv('AZURE_OPENAI_KEY')
os.environ["OPENAI_API_ENDPOINT"] = os.getenv('AZURE_OPENAI_ENDPOINT')
os.environ["OPENAI_API_VERSION"] = os.getenv('AZURE_OPENAI_API_VERSION')
os.environ["AZURE_OPENAI_DEPLOYMENT"] = os.getenv('AZURE_OPENAI_DEPLOYMENT')

# LCEL & Runnables

### Quick Notes
- Expose schematic information about their input, output and config through their `.input_schema` property, the `.output_schema` property and the `.config_schema` method
- LCEL is a declarative way to compose Runnables into chains - any chain constructed with LCEL will also always automatically have sync, async, batch, and streaming support

### Main Primitives
#### 1. RunnableSequence
- Invokes a series of Runnables sequentially
- output of the previous runnable will be input to the next runnable in the chain
- can use pipe operator to construct or by passing a list of RunnableSequence

#### 2. Runnable Parallel
- Invokes runnables concurrently, providing the same input to each. Construct using dict literal within a sequence or by passing a dict to RunnableParallel

### Additional Methods
- can be used to modify behavior such as retry policy, lifecycle listeners, make them configurable 

### Debugging
- You can use `set_debug` from `from langchain_core.globals import set_debug`. set_debug(True)
- You can pass existing or custom callbacks to any give chain too:

```from langchain_core.tracers import ConsoleCallbackHandler

chain.invoke(
    ...,
    config={'callbacks': [ConsoleCallbackHandler()]}
)```

### Passing Data through
- RunnablePassthrough allows to pass inputs unchanged or with the addition of extra keys. This typically is used in conjuction with RunnableParallel to assign data to a new key in the map.
- RunnablePassthrough() called on it’s own, will simply take the input and pass it through.

In [1]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(multiplybythree=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1},
 'extra': {'num': 1, 'multiplybythree': 3},
 'modified': 2}

- The `passed` key was called with RunnablePassthrough() and so it passed on {'num': 1}.

- In the second line with the key `extra`, we used RunnablePastshrough.assign with a lambda that multiplies the passed integer value by 3. So, `extra` was set with {'num': 1, 'mult': 3} which is the original value with the `'multiplybythree'` key added.

- Finally, we set a third key called `modified` in the map which uses a labmda to set a single value adding 1 to the num, which resulted in modified key with the value of 2.

### Run custom functions

In [ ]:
from langchain_openai import AzureChatOpenAI
# Load Azure OpenAI model
model = AzureChatOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0
)

In [20]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda


def length_function(text):
    return len(text)


def multiple_length_function(data):
    return len(data["text1"]) * len(data["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("key") | RunnableLambda(length_function),
        
        "b": {"text1": itemgetter("key"), "text2": itemgetter("value")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

#### NOTE
All inputs to these functions need to be a **SINGLE** argument. If you have a function that accepts multiple arguments, you should write a wrapper that accepts a single input and unpacks it into multiple argument.

In [21]:
chain.invoke({"key": "haha", "value": "hahaha"})

AIMessage(content='4 + 24 equals 28.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-e3ca10a9-2544-4b28-af01-5518027528f3-0', usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {}, 'output_token_details': {}})

- We use `itemgetter` to extract the input arguments
- `"a"` becomes `4` because `itemgetter` gets the value of `"haha"` and that is passed through pipe operator to the RunnableLamba which passes the value to the `length_function` which returns length of `"hahaha"` which is `6`
- `"b"` is `24` because we pass the dictionary of `{"text1": "haha", "text2": "hahaha"}` to the RunnableLambda that passes it into the `multiple_length_function`
- the dictionary of `{"a":4, "b": 24}` is then passed as output to the prompt usuing the pipe operator (`|`)
- the output of the prompt again is passed to the model
  

## Dynamic Routing based on Input

**Two ways to perform routing:**
1. Using a `RunnableBranch`.
2. Writing custom factory function that takes the input of a previous step and returns a runnable. Importantly, this should return a runnable and NOT actually execute.

### RunnableBranch

In [22]:
import os
import dotenv

dotenv.load_dotenv()
os.environ["AZURE_OPENAI_KEY"] = os.getenv("AZURE_OPENAI_KEY")

In [23]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [24]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `Bodybuilding`, `Jungian Psychology`, or `Other`.

        Do not respond with more than one word.

        <question>
        {question}
        </question>

        Classification:"""
    )
    | model
    | StrOutputParser()
)

In [25]:
chain.invoke({"question": "How do I bench press properly?"})

'Bodybuilding'

In [26]:
chain.invoke({"question": "How do I do active imagination?"})

'Jungian'

In [28]:
body_building_chain =  ( PromptTemplate.from_template(
        """You are an expert in bodybuilding. \
        Always answer questions starting with "As Dr. Mike Israetel from Renaissance Periodization told me". \
        Respond to the following question:
        
        Question: {question}
        Answer:"""
    )
    | model
)
jungian_chain = (PromptTemplate.from_template(
    """
    You are an expert in Jungian Psychology and Theory. \
    Always answer questions starting with "As Dr. C.G. Jung would say". \
    Respond to the following question:

    Question: {question}
    Answer:
    """) | model)

In [29]:
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)

In [30]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (lambda x: "bodybuilding" in x["topic"].lower(), body_building_chain),
    (lambda x: "jungian psychology" in x["topic"].lower(), jungian_chain),
    general_chain,
)

In [31]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [32]:
full_chain.invoke({"question": "how do I interpret my dreams?"})

AIMessage(content="Interpreting dreams can be a fascinating and insightful process, but it's important to remember that dream interpretation is highly subjective and can vary greatly from person to person. Here are some steps you can take to help interpret your dreams:\n\n1. **Keep a Dream Journal**: Write down your dreams as soon as you wake up. Include as many details as possible, such as people, places, emotions, and any symbols that stood out.\n\n2. **Identify Common Themes**: Look for recurring themes, symbols, or emotions in your dreams. These can provide clues about what your subconscious mind is trying to communicate.\n\n3. **Consider Your Current Life Situation**: Reflect on what is happening in your waking life. Dreams often reflect our thoughts, feelings, and concerns. Ask yourself if any part of your dream relates to your current experiences or emotions.\n\n4. **Explore Symbolism**: Many dreams contain symbols that can have personal or universal meanings. For example, water

In [33]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='2 + 2 equals 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 24, 'total_tokens': 32, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-f7863eee-b91e-43c7-b340-e18c33f4da8f-0', usage_metadata={'input_tokens': 24, 'output_tokens': 8, 'total_tokens': 32, 'input_token_details': {}, 'output_token_details': {}})

### Routing with Custom Function

In [34]:
def route(info):
    if "bodybuilding" in info["topic"].lower():
        return body_building_chain
    elif "jungian psychology" in info["topic"].lower():
        return jungian_chain
    else:
        return general_chain

In [36]:
from langchain.schema.runnable import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [37]:
full_chain.invoke({"question": "how do I grow my traps?"})

AIMessage(content='As Dr. Mike Israetel from Renaissance Periodization told me, growing your traps involves a combination of heavy compound movements and targeted isolation exercises. He recommends incorporating exercises like deadlifts, shrugs, and upright rows into your routine. Deadlifts are particularly effective because they engage the traps as stabilizers, allowing you to lift heavy weights. Shrugs, on the other hand, directly target the upper traps and can be performed with dumbbells or a barbell. Upright rows also help in hitting the traps from a different angle. Dr. Israetel emphasizes the importance of progressive overload and ensuring you are consistently increasing the weight or reps over time to stimulate muscle growth. Additionally, maintaining proper form and a full range of motion during these exercises is crucial for maximizing trap development.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 55, 'tot

In [38]:
full_chain.invoke({"question": "what does alchemy have to do with dreams?"})

AIMessage(content='Alchemy and dreams are both rich in symbolism and transformation, making them closely related in various philosophical and psychological contexts. Alchemy, historically, was the medieval precursor to modern chemistry, focused on the transmutation of base metals into noble metals like gold. However, it also had a deeply spiritual and mystical dimension, aiming for the transformation of the self and the attainment of enlightenment or the "Philosopher\'s Stone."\n\nDreams, on the other hand, are a natural phenomenon where the mind processes experiences, emotions, and subconscious thoughts during sleep. In the realm of psychology, particularly in the work of Carl Jung, dreams are seen as a way for the unconscious mind to communicate with the conscious self, often using symbolic language.\n\nJungian psychology draws a strong parallel between alchemy and dreams. Jung believed that the alchemical process was a metaphor for personal transformation and individuation—the proce

## Binding runtime arguments
- we can use `Runnable.bind()` to pass arguments as constants so that we can have access to them even within a runnable sequence where the argument is not part of the output of preceding runnables in the sequence

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out 4 flashcard questions with their options based on the topic given below.. Use the format\n\QUESTIONS:...\nOPTIONS:...\nSOLUTION:...\n\n",
        ),
        ("human", "{topic}"),
    ]
)
runnable = (
    {"topic": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("the cuban missile crisis"))

\QUESTIONS: When did the Cuban Missile Crisis take place?
OPTIONS:
A) 1959
B) 1962
C) 1965
D) 1969
SOLUTION: B) 1962

\QUESTIONS: Which two countries were directly involved in the Cuban Missile Crisis?
OPTIONS:
A) United States and China
B) United States and Soviet Union
C) United States and North Korea
D) United States and Vietnam
SOLUTION: B) United States and Soviet Union

\QUESTIONS: Who was the President of the United States during the Cuban Missile Crisis?
OPTIONS:
A) Dwight D. Eisenhower
B) Lyndon B. Johnson
C) John F. Kennedy
D) Richard Nixon
SOLUTION: C) John F. Kennedy

\QUESTIONS: What was the primary reason for the Cuban Missile Crisis?
OPTIONS:
A) The Soviet Union's installation of nuclear missiles in Cuba
B) The United States' invasion of Cuba
C) The Cuban government's nationalization of American businesses
D) The Soviet Union's blockade of Berlin
SOLUTION: A) The Soviet Union's installation of nuclear missiles in Cuba


In [44]:
runnable = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("the cuban missile crisis"))

Sure, here are some flashcard questions on the topic of the Cuban Missile Crisis:

1. What was the Cuban Missile Crisis?
2. When did the Cuban Missile Crisis take place?
3. Which two superpowers were involved in the Cuban Missile Crisis?
4. What event triggered the Cuban Missile Crisis?
5. Who was the President of the United States during the Cuban Missile Crisis?
6. Who was the leader of the Soviet Union during the Cuban Missile Crisis?
7. What was the main reason the Soviet Union placed missiles in Cuba?
8. How did the United States discover the presence of Soviet missiles in Cuba?
9. What was the U.S. response to the discovery of missiles in Cuba?
10. What was the role of the United Nations during the Cuban Missile Crisis?
11. How did the Cuban Missile Crisis end?
12. What were the terms of the agreement that ended the Cuban Missile Crisis?
13. What was the significance of the naval blockade during the Cuban Missile Crisis?
14. How close did the world come to nuclear war during the 

### Attach OpenAI Functions

In [42]:
function = {
    "name": "return_questions",
    "description": "extracts questions from raw text",
    "parameters": {
        "type": "object",
        "properties": {
            "raw_text": {
                "type": "string",
                "description": "The raw text of flashcard questions",
            },
            "questions": {
                "type": "array",
                "description": "array of questions",
                "items": {
                    "type": "string",
                    "description": "question string"
                }
            },
        },
        "required": ["equation", "solution"],
    },
}

In [46]:
# Need gpt-4 to solve this one correctly
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out flashcard questions to the following topic then return the list of questions.",
        ),
        ("human", "{topic}"),
    ]
)
runnable = {"topic": RunnablePassthrough()} | prompt | model
runnable.invoke("world war 2")

AIMessage(content="Sure, here are some flashcard questions related to World War II:\n\n1. What were the main causes of World War II?\n2. When did World War II begin and end?\n3. Which event is commonly considered the starting point of World War II?\n4. Who were the Axis Powers in World War II?\n5. Who were the Allied Powers in World War II?\n6. What was the significance of the Treaty of Versailles in the context of World War II?\n7. What was the Blitzkrieg strategy, and which country used it?\n8. What was the significance of the Battle of Britain?\n9. What was Operation Barbarossa?\n10. What was the significance of the attack on Pearl Harbor?\n11. Who were the leaders of the major countries involved in World War II (e.g., USA, UK, USSR, Germany, Japan, Italy)?\n12. What was the Holocaust?\n13. What was the significance of D-Day?\n14. What were the major conferences held by the Allies during the war (e.g., Yalta, Potsdam)?\n15. What was the Manhattan Project?\n16. What were the outcomes

## Fallbacks
- we can use fallbacks at the runnable level

In [47]:
from langchain.chat_models import ChatAnthropic, ChatOpenAI
import dotenv
dotenv.load_dotenv()


True

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

In [49]:
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")
response = httpx.Response(200, request=request)
error = RateLimitError("rate limit", response=response, body="")

In [ ]:
# Note that we set max_retries = 0 to avoid retrying on RateLimits, etc
openai_llm = ChatOpenAI(max_retries=0)
anthropic_llm = ChatAnthropic()
llm = openai_llm.with_fallbacks([anthropic_llm])

In [ ]:
# Let's use just the OpenAI LLm first, to show that we run into an error
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

Hit error


In [ ]:
# Now let's try with fallbacks to Anthropic
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

content=' I don\'t have enough context to determine the chicken\'s true motivation, but the classic punchline is: "To get to the other side!" It\'s an anti-joke playing on the double meaning of "the other side" referring to either the other side of the road, or the afterlife. Without more details, I\'d just be clucking in the dark trying to explain this chicken\'s reasoning.'


In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"animal": "kangaroo"}))
    except RateLimitError:
        print("Hit error")

content=" I don't know, why did the kangaroo cross the road?"


In [ ]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

In [ ]:
# Now lets create a chain with the normal OpenAI model
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
llm = OpenAI()
good_chain = prompt | llm

In [ ]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

"\n\nAnswer: The turtle crossed the road to get to the other side! That's a great question, by the way - you have a great sense of humor!"